In [1]:
!pip install Flask Django numpy keras tensorflow scikit-learn pandas requests


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\HamzaBaig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\HamzaBaig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\HamzaBaig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\HamzaBaig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pip\_vendor\cachecontrol\filewrapper.py", lin


  Using cached Django-4.2.11-py3-none-any.whl.metadata (4.2 kB)
  Using cached keras-3.1.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached tensorflow-2.16.1-cp39-cp39-win_amd64.whl.metadata (3.5 kB)
  Using cached asgiref-3.8.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached sqlparse-0.4.4-py3-none-any.whl.metadata (4.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.7-py3-none-any.whl.metadata (246 bytes)
  Using cached h5py-3.10.0-cp39-cp39-win_amd64.whl.metadata (2.5 kB)
     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     ---------------- --------------------- 20.5/46.2 kB 682.7 kB/s eta 0:00:01
     --------------------------------- ---- 41.0/46.2 kB 393.8 kB/s eta 0:00:01
     --------------------------------- ---- 41.0/46.2 kB 393.8 kB/s eta 0:00:01
     --------------------------------- ---- 41.0/46.2 kB 393.8 kB/s eta 0:00:01
     -------------

In [1]:
from flask import Flask, jsonify
from django.http import JsonResponse
import numpy as np
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import requests

app = Flask(__name__)

def benchmark(data):
    df = data
    closing_data = df['close'].values.reshape(-1, 1)
    other_data = df.drop(columns=['close']).values

    scaler_close = MinMaxScaler(feature_range=(0, 1))
    closing_data_scaled = scaler_close.fit_transform(closing_data)

    scaler_other = MinMaxScaler(feature_range=(0, 1))
    other_data_scaled = scaler_other.fit_transform(other_data)
    dataset_scaled = np.concatenate((other_data_scaled, closing_data_scaled), axis=1)

    model = Sequential()
    model.add(
        Bidirectional(LSTM(30, input_shape=(30, df.shape[1]),
                          return_sequences=True),
                      merge_mode='sum',
                      weights=None,
                      input_shape=(30, df.shape[1])))
    model.add(LSTM(10, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(4, return_sequences=False))
    model.add(Dense(4, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    model.load_weights("Desktop/FYP/weights.best.hdf5")

    pred = model.predict(np.reshape(dataset_scaled, (1, 30, df.shape[1])))
    pred_close = scaler_close.inverse_transform(pred.reshape(-1, 1))
    next_interval_predicted_closing = pred_close[-1][0]

    return next_interval_predicted_closing

@app.route('/predict-next-interval-close', methods=['GET'])
def predict_next_interval_close():
    requestResponse = requests.get("https://api.tiingo.com/tiingo/fx/eurusd/prices?resampleFreq=15min&token=53641a0ef9c76650ceaf3c7c4bc1e106e05ff48d")
    data = requestResponse.json()
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df['avg_price'] = (df['low'] + df['high']) / 2
    df['ohlc_price'] = (df['low'] + df['high'] + df['open'] + df['close']) / 4
    df['oc_diff'] = df['open'] - df['close']
    df.drop(columns=['ticker'], inplace=True)
    df.index = df.index.strftime('%Y-%m-%d %H:%M:%S')
    last_30_rows = df.tail(30)
    
    next_interval_closing = benchmark(last_30_rows)
    next_interval_closing_json = "{:.6f}".format(next_interval_closing)
    return jsonify({'predicted_next_interval_close': next_interval_closing_json})

if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'django'